<a id="1.1"></a>
<h3 style="background-color:skyblue;font-family:newtimeroman;font-size:200%;text-align:center">Inspiration: Ireland Apology to unwed mothers</h3>

By Australian Associated Press

The Irish government is poised to make a formal apology for abuses in church-run homes for unmarried women and their babies, where thousands of infants died and were sometimes buried in mass graves.

Prime Minister Micheal Martin is expected to apologise on behalf of the Irish state later in the week.

The Sunday Independent newspaper, which disclosed leaked details of the report, said it found that 9000 children died in 18 different homes during the 20th century.

One in seven of all children born in the homes died, far above Ireland's nationwide infant mortality rate.

Church-run homes in Ireland housed orphans, unmarried pregnant women and their babies for most of the 20th century.
https://www.greatlakesadvocate.com.au/story/7083842/ireland-apology-to-unwed-mothers/?cs=10935

![](https://www.irishcentral.com/uploads/article/118315/cropped_Mother_Baby_home_children_nuns.jpg?t=1597916262)irishcentral.com

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

The institutions have been subject to intense public scrutiny since historian Catherine Corless in 2014 tracked down death certificates for nearly 800 children who died at the former Bon Secours Mother and Baby Home in Tuam, County Galway - but could only find a burial record for one child.

Investigators later found a mass grave containing the remains of babies and young children in an underground sewage structure on the grounds of the home, which was run by an order of Catholic nuns and closed in 1961.

The last of Ireland's mother-and-baby homes did not shut down until the late 1990s.

The inquiry is part of a process of reckoning in overwhelmingly Roman Catholic Ireland to come to terms with a history of abuses in church-run institutions, including the shunning and shaming of unwed mothers, many of whom were pressured into giving up their babies for adoption.

https://www.greatlakesadvocate.com.au/story/7083842/ireland-apology-to-unwed-mothers/?cs=10935

In [ ]:
nRowsRead = 1000 # specify 'None' if want to read whole file
df = pd.read_csv('../input/cusersmarildownloadsfostercsv/foster.csv', delimiter=';', encoding = "utf8", nrows = nRowsRead)
df.dataframeName = 'foster.csv'
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')
df.head()

In [ ]:
#Chart by Roshan Sharma https://www.kaggle.com/roshansharma/immigration-to-canada

import plotly_express as px

fig = px.bar_polar(df, r="RegionID", theta="AreaID",color="Area", template="plotly_dark",
            color_discrete_sequence= px.colors.sequential.Plasma[-2::-1])
fig.show()

#Codes by AndresHG https://www.kaggle.com/andreshg/mushroom-s-h2o-automl-and-clasic-models-auc-1-0/notebook

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt

from pandas_profiling import ProfileReport
import seaborn as sns
from sklearn import metrics

from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [ ]:
import h2o

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSVa9preDPmO6qLIYPUXXyVmKcWPifMpp7qxg&usqp=CAU)h2oai-web.herokuapp.com

In [ ]:
foster_report = ProfileReport(df)

In [ ]:
foster_report

In [ ]:
# we can delete constant variables
df = df.drop('Year', axis=1)
df.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(df['RegionID'])

In [ ]:
model_df = df.copy()
model_df['RegionID'] = y
model_df.head()

In [ ]:
from h2o.estimators import H2OXGBoostEstimator

In [ ]:
h2o.init()

In [ ]:
hf = h2o.H2OFrame(model_df)

In [ ]:
hf['RegionID'] = hf['RegionID'].asfactor()
predictors = hf.drop('RegionID').columns
response = 'RegionID'

In [ ]:
# Split into train and test
train, valid = hf.split_frame(ratios=[.8], seed=1234)

In [ ]:
# Build and train the model:
foster_xgb = H2OXGBoostEstimator(booster='dart',
                                  normalize_type="tree",
                                  seed=1234,
                                  nfolds=5,
                                  )

In [ ]:
# Train the model
foster_xgb.train(x=predictors,
                  y=response,
                  training_frame=train,
                  validation_frame=valid)

In [ ]:
foster_xgb.metric(metric='accuracy', valid=True)

Too good to be true. 1st Error: AttributeError: type object 'ModelBase' has no attribute 'metric'

In [ ]:
foster_xgb.model_performance(valid=True)

In [ ]:
foster_xgb.varimp_plot(num_of_features=10)

In [ ]:
foster_xgb.shap_summary_plot(valid)

Failed: Job with key %s failed with an exception: %s" % -->

In [ ]:
from h2o.automl import H2OAutoML

In [ ]:
aml = H2OAutoML(
    max_models=20,
    max_runtime_secs=300,
    seed=1
)

In [ ]:
# Train the model
aml.train(x=predictors,
        y=response,
        training_frame=train,
        validation_frame=valid
)

In [ ]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

Below, I don't have AUC, therefore I changed to the only column above that didn't result in Error.

In [ ]:
lb = lb.as_data_frame()
lb['model_type'] = lb['model_id'].apply(lambda x: x.split('_')[0])
fig = px.bar(
    lb, 
    x='model_id',
    y='mse',
    color='model_type'
)
fig.update_yaxes(range=[0.999, 1])
fig.show()

ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of 'model_id', 'mean_per_class_error', 'logloss', 'rmse', 'mse', 'model_type' but received: auc.  Then I changed to the only that doesn't result in error. (mse)

I don't Have ACCURACY, so I changed to mse (though I left print Accuracy)

In [ ]:
print('The model performance in Accuracy: {}'.format(aml.leader.mse(valid=True)))
print('The model performance in AUC: {}'.format(aml.leader.mse(valid=True)))

In [ ]:
aml.leader.varimp_plot(num_of_features=10)

#Classification Models Comparisson

In [ ]:
# Label encoding y - dependent variable
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(df['Area'])

In [ ]:
# One hot encoding independent variable x

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
x = ohe.fit_transform(df.drop('Area', axis=1)).toarray()
x.shape

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit_transform(x)

In [ ]:
covariance = pca.get_covariance()
explained_variance = pca.explained_variance_

In [ ]:
fig = go.Figure(go.Bar(
    x=list(range(30)), 
    y=explained_variance[:30],
))
fig.update_layout(
    xaxis_title='Principal components',
    yaxis_title='Explained variance ratio',
    legend_title='individual explained variance',
)
fig.show()

In [ ]:
pca = PCA(n_components=9)
x_pca = pca.fit_transform(x)

In [ ]:
x_pca.shape

In [ ]:
# Splitting the dataset into training set and test set
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_pca, y, test_size = 0.2, random_state=1234)

In [ ]:
# Create a list for the accuracy of all the models we are going to see
acscore = []

In [ ]:
# Training the Logistic Regression Model on the Training set
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(random_state=1234)
classifier.fit(x_train, y_train)

In [ ]:
# Predicting the test set
y_pred = classifier.predict(x_test)

# Making the confusion matrix and calculating accuracy score
accuracy = metrics.accuracy_score(y_test, y_pred)
acscore.append(accuracy)
print(accuracy)

I don't have Accuracy. 

In [ ]:
# Training the Naive Bayes Classification model
from sklearn.naive_bayes import GaussianNB

classifier = GaussianNB()
classifier.fit(x_train, y_train)

In [ ]:
# Predicting the test set
y_pred = classifier.predict(x_test)

# Making the confusion matrix and calculating accuracy score
accuracy = metrics.accuracy_score(y_test, y_pred)
acscore.append(accuracy)
print(accuracy)

No Accuracy

In [ ]:
# Training the XGBoost Classification model
from xgboost import XGBClassifier

classifier = XGBClassifier()
classifier.fit(x_train,y_train)

In [ ]:
# Predicting the test set
y_pred = classifier.predict(x_test)

# Making the confusion matrix and calculating accuracy score
accuracy = metrics.accuracy_score(y_test, y_pred)
acscore.append(accuracy)
print(accuracy)

No Accuracy

In [ ]:
# Training the Random Forest Classification model
from sklearn.ensemble import RandomForestClassifier

# Lets use a standard 60 n_erstimatoprs
classifier = RandomForestClassifier(criterion='entropy', random_state=1234, n_estimators=60)
classifier.fit(x_train, y_train)

In [ ]:
# Predicting the test set
y_pred = classifier.predict(x_test)

# Making the confusion matrix and calculating accuracy score
accuracy = metrics.accuracy_score(y_test, y_pred)
acscore.append(accuracy)
print(accuracy)

No Accuracy.

In [ ]:
models_names = ['LogisticRegression', 'NaiveBayes', 'XGBoost', 'RandomForest']

In [ ]:
fig = go.Figure(go.Bar(
    x=models_names,
    y=acscore,
))
fig.show()

That's what happen with no Accuracy, no AUC, and No skills.

It was like playing BATTLESHIP: Water! With H2O I only got Water in my BattleShip. 


H2O has and awesome performance and is really fast. Also, you can check the Sparkling Water parallelization for bigger datasets: H2O original documentation https://www.h2o.ai/blog/how-sparkling-water-brings-h2o-to-spark/